<a href="https://colab.research.google.com/github/MonicaCushy/CDSS/blob/main/MycoPromoter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: 重启后第一个运行 - 预先修补文件
import os

# 先挂载Drive
from google.colab import drive
drive.mount('/content/drive')

# 安装依赖
!pip install transformers datasets accelerate einops -q

# 下载模型（但不加载到内存）
print("📥 下载模型文件...")
!python -c "from huggingface_hub import snapshot_download; snapshot_download('zhihan1996/DNABERT-2-117M')"

# 找到文件路径
import subprocess
result = subprocess.run(['find', '/root/.cache', '-name', 'flash_attn_triton.py'], capture_output=True, text=True)
flash_files = [p.strip() for p in result.stdout.strip().split('\n') if p.strip()]
result = subprocess.run(['find', '/root/.cache', '-name', 'bert_layers.py'], capture_output=True, text=True)
bert_files = [p.strip() for p in result.stdout.strip().split('\n') if p.strip()]

print(f"找到 flash_attn_triton.py: {flash_files}")
print(f"找到 bert_layers.py: {bert_files}")

# 用纯PyTorch实现替换flash_attn_triton.py
fake_flash_attn = '''
import torch
import torch.nn.functional as F
import math

def flash_attn_qkvpacked_func(qkv, bias=None, dropout_p=0.0, softmax_scale=None, causal=False, *args, **kwargs):
    batch, seqlen, three, nheads, headdim = qkv.shape
    q, k, v = qkv.unbind(dim=2)
    q, k, v = q.transpose(1,2), k.transpose(1,2), v.transpose(1,2)
    scale = softmax_scale or (1.0 / math.sqrt(headdim))
    attn = torch.matmul(q, k.transpose(-2,-1)) * scale
    if bias is not None: attn = attn + bias
    if causal:
        mask = torch.triu(torch.ones(seqlen, seqlen, device=qkv.device, dtype=torch.bool), diagonal=1)
        attn = attn.masked_fill(mask, float('-inf'))
    attn = F.softmax(attn, dim=-1, dtype=torch.float32).to(qkv.dtype)
    if dropout_p > 0 and torch.is_grad_enabled(): attn = F.dropout(attn, p=dropout_p)
    return torch.matmul(attn, v).transpose(1,2)

flash_attn_func = flash_attn_qkvpacked_func
flash_attn_kvpacked_func = flash_attn_qkvpacked_func
'''

for f in flash_files:
    print(f"✏️ 替换: {f}")
    with open(f, 'w') as fp:
        fp.write(fake_flash_attn)

print("✅ 修补完成！现在可以安全加载模型了")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Cell 2: 加载并测试（自动检测设备）
import torch
from transformers import AutoModel, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ 使用设备: {device}")

if device == "cpu":
    print("⚠️ GPU未启用！请点击 Runtime → Change runtime type → 选择 T4 GPU")
else:
    tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
    model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to(device)

    inputs = tokenizer("ACGTACGTACGT", return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**inputs)
    print(f"✅ 成功！Shape: {out.last_hidden_state.shape}")

In [ ]:
# Cell 3: 确认成功
print(f"✅ 模型加载成功！")
print(f"输出类型: {type(out)}")
print(f"输出长度: {len(out)}")
print(f"Hidden states shape: {out[0].shape}")

# 这就是我们需要的embedding
print(f"\n🎉 DNABERT-2 已经可以正常工作了！")

In [ ]:
# Cell 4: 检查数据文件
import os

# 检查可能的路径
paths_to_check = [
    "/content/drive/MyDrive/MycoPromoter/pretrain_corpus.txt",
    "/content/drive/MyDrive/pretrain_corpus.txt",
]

for p in paths_to_check:
    if os.path.exists(p):
        with open(p, 'r') as f:
            lines = f.readlines()
        print(f"✅ 找到: {p}")
        print(f"   序列数量: {len(lines)}")
        print(f"   示例: {lines[0][:50].strip()}...")
        DATA_PATH = p
        break
else:
    print("❌ 没找到数据文件")
    print("\n请运行这个命令查看你的Drive内容:")
    print("!ls -la '/content/drive/MyDrive/'")

In [ ]:
# Cell 5: MLM预训练完整代码
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

# ==================== 配置 ====================
DATA_PATH = "/content/drive/MyDrive/MycoPromoter/pretrain_corpus.txt"
OUTPUT_DIR = "/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium"

BLOCK_SIZE = 128        # 序列长度
MLM_PROB = 0.15         # 掩码比例 (教授要求)
BATCH_SIZE = 32         # T4可以handle
EPOCHS = 3              # 先跑3轮看效果
LR = 5e-5               # 学习率

# ==================== 加载数据 ====================
print("📊 加载数据...")
with open(DATA_PATH, 'r') as f:
    sequences = [line.strip() for line in f if line.strip()]

print(f"   总序列数: {len(sequences)}")

# 创建dataset
dataset = Dataset.from_dict({"text": sequences})
dataset = dataset.train_test_split(test_size=0.05, seed=42)
print(f"   训练集: {len(dataset['train'])}, 验证集: {len(dataset['test'])}")

# ==================== Tokenize ====================
print("\n🔤 Tokenizing...")
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=BLOCK_SIZE,
        padding="max_length",
        return_special_tokens_mask=True
    )

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])
print(f"   ✅ Tokenization完成")

# ==================== 加载模型 ====================
print("\n🤖 加载模型...")
model = AutoModelForMaskedLM.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = model.cuda()
print(f"   参数量: {sum(p.numel() for p in model.parameters()):,}")

# ==================== 数据整理器 ====================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=MLM_PROB
)

# ==================== 训练配置 ====================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,

    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,

    learning_rate=LR,
    weight_decay=0.01,
    warmup_ratio=0.1,

    fp16=True,

    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    report_to="none",

    # 修复：禁用safetensors格式
    save_safetensors=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    data_collator=data_collator,
)

print("🚀 继续训练...")
trainer.train()

# 保存最终模型
print("\n💾 保存模型...")
model.save_pretrained(f"{OUTPUT_DIR}/final", safe_serialization=False)
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")
print(f"✅ 已保存到: {OUTPUT_DIR}/final")

In [ ]:
# Cell 1: 恢复环境 + 加载训练好的模型

# 挂载Drive
from google.colab import drive
drive.mount('/content/drive')

# 安装依赖
!pip install transformers datasets accelerate einops -q

# 检查模型文件是否存在
import os
MODEL_PATH = "/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final"

print("📁 检查保存的文件:")
if os.path.exists(MODEL_PATH):
    for f in os.listdir(MODEL_PATH):
        size = os.path.getsize(f"{MODEL_PATH}/{f}") / 1024 / 1024
        print(f"  ✅ {f} ({size:.1f} MB)")
else:
    print("❌ 模型文件夹不存在！")

In [ ]:
# Cell 2: 修补Flash Attention + 加载模型

# Step 1: 下载并修补DNABERT-2的依赖文件
print("🔧 修补Flash Attention...")
from huggingface_hub import snapshot_download
snapshot_download('zhihan1996/DNABERT-2-117M')

import subprocess
result = subprocess.run(['find', '/root/.cache', '-name', 'flash_attn_triton.py'], capture_output=True, text=True)
flash_files = [p.strip() for p in result.stdout.strip().split('\n') if p.strip()]

fake_flash_attn = '''
import torch
import torch.nn.functional as F
import math

def flash_attn_qkvpacked_func(qkv, bias=None, dropout_p=0.0, softmax_scale=None, causal=False, *args, **kwargs):
    batch, seqlen, three, nheads, headdim = qkv.shape
    q, k, v = qkv.unbind(dim=2)
    q, k, v = q.transpose(1,2), k.transpose(1,2), v.transpose(1,2)
    scale = softmax_scale or (1.0 / math.sqrt(headdim))
    attn = torch.matmul(q, k.transpose(-2,-1)) * scale
    if bias is not None: attn = attn + bias
    if causal:
        mask = torch.triu(torch.ones(seqlen, seqlen, device=qkv.device, dtype=torch.bool), diagonal=1)
        attn = attn.masked_fill(mask, float('-inf'))
    attn = F.softmax(attn, dim=-1, dtype=torch.float32).to(qkv.dtype)
    if dropout_p > 0 and torch.is_grad_enabled(): attn = F.dropout(attn, p=dropout_p)
    return torch.matmul(attn, v).transpose(1,2)

flash_attn_func = flash_attn_qkvpacked_func
flash_attn_kvpacked_func = flash_attn_qkvpacked_func
'''

for f in flash_files:
    with open(f, 'w') as fp:
        fp.write(fake_flash_attn)
    print(f"  ✅ 修补: {f}")

# Step 2: 加载模型
print("\n🤖 加载训练好的模型...")
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

MODEL_PATH = "/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final"

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

# 加载你训练的权重
state_dict = torch.load(f"{MODEL_PATH}/pytorch_model.bin", weights_only=False)
model.load_state_dict(state_dict)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"✅ 模型加载成功！设备: {device}")

# Step 3: 快速测试
test_seq = "ACGTACGT"
inputs = tokenizer(test_seq, return_tensors="pt").to(device)
with torch.no_grad():
    out = model(**inputs)
print(f"✅ 前向传播测试通过！输出shape: {out.logits.shape}")

In [ ]:
## Cell 3: 下载Shell et al. 2015数据

# 这篇文章是：
# Shell SS, et al. (2015) "Leaderless Transcripts and Small Proteins Are Common Features of the Mycobacterial Translational Landscape"
# PLOS Genetics, DOI: 10.1371/journal.pgen.1005641

# Supplementary数据在PLOS的网站上
import requests
import os

print("📥 下载Shell et al. 2015 supplementary data...")

# PLOS Genetics的supplementary table通常是S1, S2等
# 这篇文章的TSS数据在supplementary tables里

# 先试着从NCBI GEO下载（这篇文章的数据在GEO: GSE62152）
urls_to_try = [
    # 论文的supplementary
    "https://doi.org/10.1371/journal.pgen.1005641.s008",  # S1 Table
    "https://doi.org/10.1371/journal.pgen.1005641.s009",  # S2 Table
]

print("尝试直接下载可能会被重定向，让我先搜索一下具体链接...")

# 显示你现有的文件
print("\n📁 你的MycoPromoter文件夹现有文件:")
myco_path = "/content/drive/MyDrive/MycoPromoter/"
for f in os.listdir(myco_path):
    print(f"  {f}")

In [ ]:
# Cell 4: 从PLOS Genetics下载Shell数据

import requests
import pandas as pd
import os

print("📥 下载Shell et al. 2015 TSS数据...")

# Shell 2015的supplementary Excel文件
# S2 Table包含TSS位置和表达量数据
url = "https://journals.plos.org/plosgenetics/article/file?type=supplementary&id=10.1371/journal.pgen.1005641.s009"

response = requests.get(url, allow_redirects=True)

if response.status_code == 200:
    # 保存文件
    save_path = "/content/drive/MyDrive/MycoPromoter/shell_2015_s2_table.xlsx"
    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f"✅ 下载成功: {save_path}")
    print(f"   文件大小: {len(response.content)/1024:.1f} KB")

    # 尝试读取
    try:
        df = pd.read_excel(save_path)
        print(f"\n📊 数据预览:")
        print(f"   行数: {len(df)}")
        print(f"   列名: {list(df.columns)}")
        print(df.head())
    except Exception as e:
        print(f"读取Excel失败: {e}")
        print("可能需要安装openpyxl: !pip install openpyxl")
else:
    print(f"❌ 下载失败: {response.status_code}")
    print("尝试备用方案...")

In [ ]:
# Cell 5: 下载包含表达量的数据表

import requests
import pandas as pd

print("📥 下载Shell 2015的表达量数据...")

# 尝试S1 Table（通常包含RPKM）
urls = {
    "S1": "https://journals.plos.org/plosgenetics/article/file?type=supplementary&id=10.1371/journal.pgen.1005641.s008",
    "S3": "https://journals.plos.org/plosgenetics/article/file?type=supplementary&id=10.1371/journal.pgen.1005641.s010",
    "S4": "https://journals.plos.org/plosgenetics/article/file?type=supplementary&id=10.1371/journal.pgen.1005641.s011",
}

for name, url in urls.items():
    print(f"\n尝试下载 {name} Table...")
    response = requests.get(url, allow_redirects=True)

    if response.status_code == 200 and len(response.content) > 1000:
        save_path = f"/content/drive/MyDrive/MycoPromoter/shell_2015_{name.lower()}_table.xlsx"
        with open(save_path, 'wb') as f:
            f.write(response.content)

        try:
            # 尝试读取所有sheet
            xl = pd.ExcelFile(save_path)
            print(f"  ✅ {name}: {len(response.content)/1024:.1f} KB")
            print(f"     Sheets: {xl.sheet_names}")

            # 读取第一个sheet看看列名
            df = pd.read_excel(save_path, sheet_name=0)
            print(f"     列名: {list(df.columns)[:8]}...")  # 只显示前8列

            # 检查是否有RPKM相关的列
            rpkm_cols = [c for c in df.columns if 'rpkm' in str(c).lower() or 'expression' in str(c).lower() or 'count' in str(c).lower()]
            if rpkm_cols:
                print(f"     🎯 找到表达量相关列: {rpkm_cols}")
        except Exception as e:
            print(f"  读取失败: {e}")

In [ ]:
# Cell 6: 查看S3数据详情
import pandas as pd

s3_path = "/content/drive/MyDrive/MycoPromoter/shell_2015_s3_table.xlsx"

# 读取All TSSs这个sheet
df = pd.read_excel(s3_path, sheet_name="All TSSs")

print(f"📊 S3 Table - All TSSs")
print(f"行数: {len(df)}")
print(f"\n所有列名:")
for i, col in enumerate(df.columns):
    print(f"  {i+1}. {col}")

print(f"\n前5行数据:")
print(df.head())

print(f"\n表达量列的统计:")
coverage_col = [c for c in df.columns if 'coverage' in c.lower() or 'mean' in c.lower()]
if coverage_col:
    print(df[coverage_col[0]].describe())

In [ ]:
# Cell 7: 下载基因组 + 提取promoter序列
!pip install biopython -q

from Bio import Entrez, SeqIO
import pandas as pd

Entrez.email = "student@example.com"

print("下载M. tuberculosis基因组...")
handle = Entrez.efetch(db="nucleotide", id="NC_000962.3", rettype="fasta", retmode="text")
genome_record = SeqIO.read(handle, "fasta")
handle.close()
genome_seq = str(genome_record.seq)
print(f"基因组长度: {len(genome_seq)} bp")

df = pd.read_excel("/content/drive/MyDrive/MycoPromoter/shell_2015_s3_table.xlsx", sheet_name="All TSSs")
print(f"TSS数量: {len(df)}")

UPSTREAM = 150
DOWNSTREAM = 50

def extract_promoter(row, genome):
    coord = int(row["Coordinate (genome version NC_000962)"])
    strand = row["Strand"]
    if strand == "+":
        start = coord - UPSTREAM - 1
        end = coord + DOWNSTREAM
    else:
        start = coord - DOWNSTREAM - 1
        end = coord + UPSTREAM
    start = max(0, start)
    end = min(len(genome), end)
    seq = genome[start:end]
    if strand == "-":
        comp = {"A":"T", "T":"A", "G":"C", "C":"G", "N":"N"}
        seq = "".join(comp.get(b,"N") for b in reversed(seq))
    return seq

df["promoter_seq"] = df.apply(lambda r: extract_promoter(r, genome_seq), axis=1)
df["expression"] = df["Mean coverage in converted libraries"]

print("完成!")
print(df[["expression", "promoter_seq"]].head())

df.to_csv("/content/drive/MyDrive/MycoPromoter/shell_promoters_with_seq.csv", index=False)
print("已保存!")

In [ ]:
# Cell 8: Stage 2 - 训练promoter强度预测模型

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr

# 加载数据
df = pd.read_csv("/content/drive/MyDrive/MycoPromoter/shell_promoters_with_seq.csv")
print(f"数据量: {len(df)}")

# 过滤掉太短的序列
df = df[df["promoter_seq"].str.len() >= 100]
print(f"过滤后: {len(df)}")

# Log转换表达量（因为范围很大：20 ~ 120948）
df["log_expression"] = np.log1p(df["expression"])

# 归一化到0-1
min_val = df["log_expression"].min()
max_val = df["log_expression"].max()
df["normalized_expression"] = (df["log_expression"] - min_val) / (max_val - min_val)

print(f"表达量范围: {df['expression'].min():.1f} - {df['expression'].max():.1f}")
print(f"归一化后: {df['normalized_expression'].min():.3f} - {df['normalized_expression'].max():.3f}")

# 划分训练集/验证集/测试集
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"\n训练集: {len(train_df)}, 验证集: {len(val_df)}, 测试集: {len(test_df)}")

# 保存划分
train_df.to_csv("/content/drive/MyDrive/MycoPromoter/train.csv", index=False)
val_df.to_csv("/content/drive/MyDrive/MycoPromoter/val.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/MycoPromoter/test.csv", index=False)
print("数据集已保存!")

In [ ]:
# Cell 9: 定义Dataset和模型

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

class PromoterDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_length=128):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        seq = row["promoter_seq"]
        label = row["normalized_expression"]

        encoding = self.tokenizer(
            seq,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.float)
        }

# 回归模型：DNABERT-2 + 回归头
class PromoterStrengthModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.bert = base_model
        self.regressor = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()  # 输出0-1
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # 取[CLS] token的输出（序列的整体表示）
        cls_output = outputs[0][:, 0, :]  # (batch, 768)
        prediction = self.regressor(cls_output)
        return prediction.squeeze()

# 创建数据集
train_dataset = PromoterDataset("/content/drive/MyDrive/MycoPromoter/train.csv", tokenizer)
val_dataset = PromoterDataset("/content/drive/MyDrive/MycoPromoter/val.csv", tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"训练集batches: {len(train_loader)}")
print(f"验证集batches: {len(val_loader)}")

# 加载我们domain-adapted的DNABERT-2
print("\n加载模型...")
base_model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

# 加载Stage 1训练的权重
import torch
state_dict = torch.load("/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final/pytorch_model.bin", weights_only=False)
# 只加载bert部分的权重（忽略MLM head）
bert_state_dict = {k.replace("bert.", ""): v for k, v in state_dict.items() if k.startswith("bert.")}
base_model.load_state_dict(bert_state_dict, strict=False)

model

In [ ]:
# Cell 10: 修复模型定义并重新训练

import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import spearmanr
from transformers import AutoModel

# 重新定义模型（修复版）
class PromoterStrengthModel(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        # 直接加载base model，不是MLM model
        self.bert = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

        # 加载domain-adapted权重
        state_dict = torch.load(f"{model_path}/pytorch_model.bin", weights_only=False)
        bert_state_dict = {k.replace("bert.", ""): v for k, v in state_dict.items() if k.startswith("bert.")}
        self.bert.load_state_dict(bert_state_dict, strict=False)

        self.regressor = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[0][:, 0, :]  # [CLS] token
        prediction = self.regressor(cls_output)
        return prediction.squeeze()

# 创建新模型
print("加载模型...")
model = PromoterStrengthModel("/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final")
model = model.cuda()
print("模型准备完成!")

# 训练配置
EPOCHS = 10
LR = 2e-5

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)

print(f"\n开始训练... Epochs: {EPOCHS}")
print("-" * 50)

best_spearman = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        labels = batch["label"].cuda()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # 验证
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            labels = batch["label"]

            outputs = model(input_ids, attention_mask)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.numpy())

    spearman = spearmanr(all_preds, all_labels)[0]

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | Val Spearman: {spearman:.4f}")

    if spearman > best_spearman:
        best_spearman = spearman
        torch.save(model.state_dict(), "/content/drive/MyDrive/MycoPromoter/best_regression_model.pt")
        print(f"  -> 新最佳!")

print(f"\n训练完成! 最佳Spearman: {best_spearman:.4f}")

加载模型...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

bert_padding.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


flash_attn_triton.py: 0.00B [00:00, ?B/s]

Encountered exception while importing triton: No module named 'triton'


ImportError: This modeling file requires the following packages that were not found in your environment: triton. Run `pip install triton`

In [ ]:
# Cell 11: 冻结BERT，重新训练

import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import spearmanr
from transformers import AutoModel

class PromoterStrengthModelFrozen(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.bert = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

        # 加载domain-adapted权重
        state_dict = torch.load(f"{model_path}/pytorch_model.bin", weights_only=False)
        bert_state_dict = {k.replace("bert.", ""): v for k, v in state_dict.items() if k.startswith("bert.")}
        self.bert.load_state_dict(bert_state_dict, strict=False)

        # 冻结BERT所有参数！
        for param in self.bert.parameters():
            param.requires_grad = False

        # 更大的回归头
        self.regressor = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # BERT不计算梯度
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[0][:, 0, :]
        prediction = self.regressor(cls_output)
        return prediction.squeeze()

# 创建新模型
print("加载模型（BERT冻结版）...")
model = PromoterStrengthModelFrozen("/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final")
model = model.cuda()

# 只有回归头的参数会被训练
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可训练参数: {trainable_params:,} / {total_params:,}")

# 训练配置 - 更大的学习率（因为只训练回归头）
EPOCHS = 30
LR = 1e-3

criterion = nn.MSELoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

print(f"\n开始训练... Epochs: {EPOCHS}, LR: {LR}")
print("-" * 50)

best_spearman = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        labels = batch["label"].cuda()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # 验证
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            labels = batch["label"]

            outputs = model(input_ids, attention_mask)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.numpy())

    spearman = spearmanr(all_preds, all_labels)[0]

    # 每5个epoch打印一次，或者有新最佳时打印
    if (epoch + 1) % 5 == 0 or spearman > best_spearman:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | Val Spearman: {spearman:.4f}")

    if spearman > best_spearman:
        best_spearman = spearman
        torch.save(model.state_dict(), "/content/drive/MyDrive/MycoPromoter/best_frozen_model.pt")
        print(f"  -> 新最佳!")

print(f"\n训练完成! 最佳Spearman: {best_spearman:.4f}")

In [ ]:
# Cell 12: LoRA + 生物学特征

!pip install peft -q  # LoRA库

import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from scipy.stats import spearmanr
from transformers import AutoModel, AutoTokenizer
from peft import LoraConfig, get_peft_model

# ============ 生物学特征提取 ============

def extract_bio_features(seq):
    """从序列中提取生物学特征"""
    features = []

    # 1. GC含量
    gc = (seq.count('G') + seq.count('C')) / len(seq)
    features.append(gc)

    # 2. -10区 motif匹配 (TATAAT)，检查位置130-150
    minus10_region = seq[130:150] if len(seq) >= 150 else seq[-50:-30]
    minus10_consensus = "TATAAT"
    best_minus10_score = 0
    for i in range(len(minus10_region) - 6):
        score = sum(a == b for a, b in zip(minus10_region[i:i+6], minus10_consensus))
        best_minus10_score = max(best_minus10_score, score)
    features.append(best_minus10_score / 6)  # 归一化到0-1

    # 3. -35区 motif匹配 (TTGACA)，检查位置110-130
    minus35_region = seq[110:130] if len(seq) >= 130 else seq[-70:-50]
    minus35_consensus = "TTGACA"
    best_minus35_score = 0
    for i in range(len(minus35_region) - 6):
        score = sum(a == b for a, b in zip(minus35_region[i:i+6], minus35_consensus))
        best_minus35_score = max(best_minus35_score, score)
    features.append(best_minus35_score / 6)

    # 4. AT含量在-10区附近（AT-rich有助于DNA解链）
    if len(seq) >= 150:
        at_region = seq[125:155]
        at_content = (at_region.count('A') + at_region.count('T')) / len(at_region)
    else:
        at_content = (seq.count('A') + seq.count('T')) / len(seq)
    features.append(at_content)

    # 5. 简单重复序列比例
    repeats = len(re.findall(r'(.)\1{2,}', seq))  # 3个以上连续相同碱基
    features.append(min(repeats / 10, 1.0))

    return features

# 测试特征提取
test_seq = "ACGT" * 50
print(f"特征示例: {extract_bio_features(test_seq)}")
print(f"特征含义: [GC含量, -10区匹配, -35区匹配, AT含量, 重复序列]")

# ============ 带生物特征的Dataset ============

from torch.utils.data import Dataset, DataLoader
import pandas as pd

class PromoterDatasetWithFeatures(Dataset):
    def __init__(self, csv_path, tokenizer, max_length=128):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        seq = row["promoter_seq"]
        label = row["normalized_expression"]

        # Tokenize
        encoding = self.tokenizer(
            seq, truncation=True, max_length=self.max_length,
            padding="max_length", return_tensors="pt"
        )

        # 生物学特征
        bio_features = torch.tensor(extract_bio_features(seq), dtype=torch.float)

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "bio_features": bio_features,
            "label": torch.tensor(label, dtype=torch.float)
        }

# ============ LoRA + 生物特征 模型 ============

class PromoterModelLoRA(nn.Module):
    def __init__(self, model_path):
        super().__init__()

        # 加载BERT
        self.bert = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

        # 加载domain-adapted权重
        state_dict = torch.load(f"{model_path}/pytorch_model.bin", weights_only=False)
        bert_state_dict = {k.replace("bert.", ""): v for k, v in state_dict.items() if k.startswith("bert.")}
        self.bert.load_state_dict(bert_state_dict, strict=False)

        # 添加LoRA
        lora_config = LoraConfig(
            r=8,                      # LoRA秩，越小参数越少
            lora_alpha=16,            # 缩放因子
            target_modules=["Wqkv"],  # 只在attention层加LoRA
            lora_dropout=0.1,
            bias="none"
        )
        self.bert = get_peft_model(self.bert, lora_config)

        # 回归头：BERT输出(768) + 生物特征(5) = 773
        self.regressor = nn.Sequential(
            nn.Linear(768 + 5, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask, bio_features):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[0][:, 0, :]  # [CLS] token: (batch, 768)

        # 拼接BERT输出和生物特征
        combined = torch.cat([cls_output, bio_features], dim=1)  # (batch, 773)

        prediction = self.regressor(combined)
        return prediction.squeeze()

# ============ 准备数据 ============

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

train_dataset = PromoterDatasetWithFeatures("/content/drive/MyDrive/MycoPromoter/train.csv", tokenizer)
val_dataset = PromoterDatasetWithFeatures("/content/drive/MyDrive/MycoPromoter/val.csv", tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

print(f"\n数据集: 训练{len(train_dataset)}, 验证{len(val_dataset)}")

# ============ 创建模型 ============

print("\n加载LoRA模型...")
model = PromoterModelLoRA("/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final")
model = model.cuda()

# 打印可训练参数
model.bert.print_trainable_parameters()

# ============ 训练 ============

EPOCHS = 20
LR = 5e-4

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)

print(f"\n开始训练... Epochs: {EPOCHS}, LR: {LR}")
print("-" * 50)

best_spearman = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        bio_features = batch["bio_features"].cuda()
        labels = batch["label"].cuda()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, bio_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # 验证
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            bio_features = batch["bio_features"].cuda()
            labels = batch["label"]

            outputs = model(input_ids, attention_mask, bio_features)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.numpy())

    spearman = spearmanr(all_preds, all_labels)[0]

    if (epoch + 1) % 5 == 0 or spearman > best_spearman:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | Val Spearman: {spearman:.4f}")

    if spearman > best_spearman:
        best_spearman = spearman
        torch.save(model.state_dict(), "/content/drive/MyDrive/MycoPromoter/best_lora_model.pt")
        print(f"  -> 新最佳!")

print(f"\n训练完成! 最佳Spearman: {best_spearman:.4f}")
```

In [ ]:
# Cell 7: 测试训练后的模型（修复版）
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

MODEL_PATH = "/content/drive/MyDrive/MycoPromoter/dnabert2_mycobacterium/final"

print("🔄 加载训练后的模型...")

# 先检查保存了什么文件
import os
print("保存的文件:")
for f in os.listdir(MODEL_PATH):
    print(f"  {f}")

# tokenizer用原版的，模型用训练后的
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

# 加载本地模型权重
model = AutoModelForMaskedLM.from_pretrained(
    "zhihan1996/DNABERT-2-117M",  # 用原版配置
    trust_remote_code=True
)
# 加载我们训练的权重
import torch
state_dict = torch.load(f"{MODEL_PATH}/pytorch_model.bin", weights_only=False)
model.load_state_dict(state_dict)
model = model.cuda()
model.eval()

print("✅ 模型加载成功！")

# 测试
test_seq = "ACGT" + tokenizer.mask_token + "ACGT"
inputs = tokenizer(test_seq, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model(**inputs)

mask_idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
logits = outputs.logits[0, mask_idx[0]]
top5 = torch.topk(logits, 5)

print(f"\n🧪 测试输入: {test_seq}")
print("Top 5 预测:")
for i, (score, idx) in enumerate(zip(top5.values, top5.indices)):
    token = tokenizer.decode([idx])
    print(f"  {i+1}. '{token}' (score: {score:.2f})")

In [ ]:
# Cell 1: 检查现有数据

# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# 加载我们之前处理好的数据
df = pd.read_csv('/content/drive/MyDrive/MycoPromoter/shell_promoters_with_seq.csv')

print(f"数据量: {len(df)}")
print(f"\n列名: {df.columns.tolist()}")
print(f"\n前几行:")
print(df.head())